In [72]:
# import des bibliothèques requises
import openai
import requests

# données d'identification et d'authentification openAI
openai.organization = "org-n6X2zCb7o31ARXwZnVkEeopQ"
openai.api_key = os.getenv("OPENAI_API_KEY")


In [80]:
# le moteur IA utilisé dans ce programme
engine = "text-davinci-002"

# la liste suivante est ici hardcodée mais sera générée depuis une liste d'ingrédients suggérés par l'interface graphique (frontend)
ingredients = ["beef","ginger","bell peppers","oregano"]

# à partir de la liste d'ingrédients, génération d'une chaîne de caractères qui s'intègrera naturellement dans la question posée
formatted_ingredients = "\n- " + "\n- ".join(ingredients)

# la question qui sera posée au modèle openAI
question_asked_to_openAI_completions = "Provide a cooking recipe based on the following ingredients:\n" + formatted_ingredients + "\n\nThe response should include the recipe's name, the recipe's ingredients, and the recipe's instructions."

# un dictionnaire contenant les arguments à passer à l'endpoint "completions" via la méthode "openai.Completion.create" (SDK openAI)
completion_arguments = {
  "engine" : engine,
  "prompt" : question_asked_to_openAI_completions,
  "temperature" : 0.7,
  "top_p" : 1,
  "max_tokens" : 300,
  "frequency_penalty" : 0,
  "presence_penalty" : 0
  }

completion_arguments


{'engine': 'text-davinci-002',
 'prompt': "Provide a cooking recipe based on the following ingredients:\n\n- beef\n- ginger\n- bell peppers\n- oregano\n\nThe response should include the recipe's name, the recipe's ingredients, and the recipe's instructions.",
 'temperature': 0.7,
 'top_p': 1,
 'max_tokens': 300,
 'frequency_penalty': 0,
 'presence_penalty': 0}

In [83]:
response = openai.Completion.create(
  engine = completion_arguments["engine"],
  prompt =  completion_arguments["prompt"],
  temperature = completion_arguments["temperature"],
  top_p = completion_arguments["top_p"],
  max_tokens = completion_arguments["max_tokens"],
  frequency_penalty = completion_arguments["frequency_penalty"],
  presence_penalty = completion_arguments["presence_penalty"])

print(response["choices"][0]["text"][2:])

-

Stir-Fried Beef with Ginger and Bell Peppers

Ingredients:

- 1 pound beef, thinly sliced
- 2 tablespoons vegetable oil
- 1 thumb-sized piece of ginger, peeled and minced
- 2 bell peppers, thinly sliced
- 1 teaspoon dried oregano
- 1 teaspoon salt
- 1/2 teaspoon black pepper

Instructions:

1. Heat a large skillet or wok over high heat.

2. Add the oil and ginger, and stir-fry for 30 seconds.

3. Add the beef and stir-fry for 2 minutes, or until browned.

4. Add the bell peppers and oregano, and stir-fry for 2 minutes.

5. Season with salt and black pepper, and stir-fry for 1 more minute.

6. Serve with rice.


In [88]:
print(response["choices"][0]["text"][5:])

Stir-Fried Beef with Ginger and Bell Peppers

Ingredients:

- 1 pound beef, thinly sliced
- 2 tablespoons vegetable oil
- 1 thumb-sized piece of ginger, peeled and minced
- 2 bell peppers, thinly sliced
- 1 teaspoon dried oregano
- 1 teaspoon salt
- 1/2 teaspoon black pepper

Instructions:

1. Heat a large skillet or wok over high heat.

2. Add the oil and ginger, and stir-fry for 30 seconds.

3. Add the beef and stir-fry for 2 minutes, or until browned.

4. Add the bell peppers and oregano, and stir-fry for 2 minutes.

5. Season with salt and black pepper, and stir-fry for 1 more minute.

6. Serve with rice.
